In [121]:
import torch
from torch_geometric.datasets import Planetoid
from hivegraph.contrastive.grace import GRACE
from sklearn.linear_model import LogisticRegression
from util import * 
from torch_geometric.utils import to_dense_adj

In [108]:
data_splits = [torch.load(f"data_splits\\cora_splits\\split_{i}.pt") for i in range(10)]
dataset_o = data_splits[0].to('cuda')

C:\Users\banfi\AppData\Local\Temp\ipykernel_18800\983164769.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_splits = [torch.load(f"data_splits\\cora_splits\\split_{

In [109]:
model = GRACE(num_features=dataset_o.num_features,hidden=128, num_layers=2, drop_edge_rate_1=0.3,drop_edge_rate_2=0.3,drop_feature_rate_1=0.3,drop_feature_rate_2=0.3).to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.01)

In [110]:
for epoch in range(200):
    opt.zero_grad()
    loss = model.train_step(dataset_o.x,dataset_o.edge_index)
    print(loss)
    loss.backward()
    opt.step()

tensor(8.5764, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(8.5720, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.9201, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(8.0819, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.7646, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.6067, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.5567, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.5274, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.4740, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.4233, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.4117, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.4088, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.4497, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.3781, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.3651, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.3269, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.3119, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.3144, device='cuda:0',

In [111]:
out = model(dataset_o.x,dataset_o.edge_index)

In [112]:
BUDGET = 150
dataset = dataset_o.clone()
for b in range(BUDGET):
    predictor = LogisticRegression()
    predictor.fit(out[dataset.train_mask].detach().cpu().numpy(), dataset.y[dataset.train_mask].detach().cpu().numpy())
    acc = predictor.score(out[dataset.test_mask].detach().cpu().numpy(), dataset.y[dataset.test_mask].detach().cpu().numpy())
    pred_log_probas = predictor.predict_log_proba(out.detach().cpu().numpy())
    entropies = calculate_entropy(torch.tensor(pred_log_probas))
    pool_indices = get_mask_indices(dataset.train_pool).cpu()
    chosen_node_ix = torch.argmax(entropies[pool_indices])
    chosen_node = pool_indices[chosen_node_ix]
    dataset.train_pool[chosen_node] = False
    dataset.train_mask[chosen_node] = True
    predictor = LogisticRegression()
    print(f"Budget {b} - Accuracy: {acc}")
    

Budget 0 - Accuracy: 0.388
Budget 1 - Accuracy: 0.437
Budget 2 - Accuracy: 0.502
Budget 3 - Accuracy: 0.533
Budget 4 - Accuracy: 0.586
Budget 5 - Accuracy: 0.599
Budget 6 - Accuracy: 0.61
Budget 7 - Accuracy: 0.629
Budget 8 - Accuracy: 0.637
Budget 9 - Accuracy: 0.666
Budget 10 - Accuracy: 0.659
Budget 11 - Accuracy: 0.687
Budget 12 - Accuracy: 0.725
Budget 13 - Accuracy: 0.724
Budget 14 - Accuracy: 0.711
Budget 15 - Accuracy: 0.71
Budget 16 - Accuracy: 0.708
Budget 17 - Accuracy: 0.699
Budget 18 - Accuracy: 0.69
Budget 19 - Accuracy: 0.69
Budget 20 - Accuracy: 0.689
Budget 21 - Accuracy: 0.685
Budget 22 - Accuracy: 0.704
Budget 23 - Accuracy: 0.712
Budget 24 - Accuracy: 0.716
Budget 25 - Accuracy: 0.72
Budget 26 - Accuracy: 0.754
Budget 27 - Accuracy: 0.746
Budget 28 - Accuracy: 0.738
Budget 29 - Accuracy: 0.742
Budget 30 - Accuracy: 0.733
Budget 31 - Accuracy: 0.733
Budget 32 - Accuracy: 0.747
Budget 33 - Accuracy: 0.744
Budget 34 - Accuracy: 0.74
Budget 35 - Accuracy: 0.745
Budget 3

In [127]:
BUDGET = 150
dataset = dataset_o.clone()
for b in range(BUDGET):
    
    predictor = LogisticRegression()
    predictor.fit(out[dataset.train_mask].detach().cpu().numpy(), dataset.y[dataset.train_mask].detach().cpu().numpy())
    acc = predictor.score(out[dataset.test_mask].detach().cpu().numpy(), dataset.y[dataset.test_mask].detach().cpu().numpy())
    
    pool_indices = get_mask_indices(dataset.train_pool).cpu()

    dist_matrix = torch.cdist(out[dataset.train_pool], out[dataset.train_pool])
    adj = to_dense_adj(dataset.edge_index)[0][dataset.train_pool][:,dataset.train_pool]
    adj = adj.cuda()
    dist_matrix = dist_matrix * adj
    
    
    max_dist = dist_matrix.max(dim=1).values
    max_dist = max_dist / adj.sum(dim=1)
    min_max_dist = torch.argmin(max_dist)
    chosen_node = pool_indices[min_max_dist]

    
    dataset.train_pool[chosen_node] = False
    dataset.train_mask[chosen_node] = True
    # predictor = LogisticRegression()
    print(f"Budget {b} - Accuracy: {acc}")

Budget 0 - Accuracy: 0.388
Budget 1 - Accuracy: 0.379
Budget 2 - Accuracy: 0.511
Budget 3 - Accuracy: 0.508
Budget 4 - Accuracy: 0.491
Budget 5 - Accuracy: 0.484
Budget 6 - Accuracy: 0.477
Budget 7 - Accuracy: 0.443
Budget 8 - Accuracy: 0.54
Budget 9 - Accuracy: 0.546
Budget 10 - Accuracy: 0.52
Budget 11 - Accuracy: 0.533
Budget 12 - Accuracy: 0.545
Budget 13 - Accuracy: 0.545
Budget 14 - Accuracy: 0.561
Budget 15 - Accuracy: 0.561
Budget 16 - Accuracy: 0.58
Budget 17 - Accuracy: 0.58
Budget 18 - Accuracy: 0.618
Budget 19 - Accuracy: 0.618
Budget 20 - Accuracy: 0.59
Budget 21 - Accuracy: 0.594
Budget 22 - Accuracy: 0.595
Budget 23 - Accuracy: 0.596
Budget 24 - Accuracy: 0.597
Budget 25 - Accuracy: 0.57
Budget 26 - Accuracy: 0.566
Budget 27 - Accuracy: 0.589
Budget 28 - Accuracy: 0.578
Budget 29 - Accuracy: 0.578
Budget 30 - Accuracy: 0.59
Budget 31 - Accuracy: 0.593
Budget 32 - Accuracy: 0.611
Budget 33 - Accuracy: 0.627
Budget 34 - Accuracy: 0.629
Budget 35 - Accuracy: 0.644
Budget 36